In [ ]:
'''
PREREQS/README

1.register for free trial here, gives thousands of free calls: api.webscrapingapi.com

2.0. learn how ebaysdk (this app uses only the trading api) works 
https://github.com/timotheus/ebaysdk-python

2.1.register for developers program here and link account: 
https://developer.ebay.com/

2.2 create your app in the portal
https://developer.ebay.com/my/keys
then generate user token by logging into your ebay selling account

3.set up your ebay.yaml in the same directory as this file
template found here: 
https://github.com/timotheus/ebaysdk-python/blob/master/ebay.yaml

'''

import requests
from urllib.parse import urlencode

import http.client

conn = http.client.HTTPSConnection("api.webscrapingapi.com")

web_scraping_api_key = 'INSERT WEBSCRAPINGAPI KEY HERE'


def get_htmls(cert):
    print("Getting HTMLS...")
        
    conn.request("GET", "/v1?api_key=" + web_scraping_api_key + "&device=desktop&proxy_type=datacenter&render_js=1&url=https%3A%2F%2Fwww.psacard.com%2Fcert%2F"+ str(cert) +"&wait_for=1&wait_until=domcontentloaded")
    res = conn.getresponse()
    data = res.read()
    data = data.decode("utf-8")
    response = data
        
    return [response]




In [ ]:
from bs4 import BeautifulSoup

def extract_info(responses):
    print("Extracting Info...")
    cards = {}
    for response in responses:
        html = response
        soup = BeautifulSoup(html, "html.parser")
        #print(soup.find_all("th"))
        k = 0

        values = soup.find_all("td")
        keys = soup.find_all("th")

        info_dict = {}

        for i in range(0,len(keys)):
            info_dict[keys[i].text] = values[i].text
            
        front = soup.find("div",{"id": "certImgFront"}).find('a', href=True)['href']
        back = soup.find("div",{"id": "certImgBack"}).find('a', href=True)['href']
        info_dict['Front'] = soup.find("div",{"id": "certImgFront"}).find('a', href=True)['href']
        info_dict['Back'] = soup.find("div",{"id": "certImgBack"}).find('a', href=True)['href']
        
    
    return info_dict



In [ ]:
def get_listing_title(cert):
    responses = ''
    attempts = 0
    cards = ''
    while attempts < 15:
        try:            
            responses = get_htmls(str(cert))
            cards = extract_info(responses)
            break
        except:
            attempts += 1
    grade = cards['Grade'].split(' ')[-1] 
    title = 'PSA '
    title += grade 
    title += ' ' 
    title += cards['Player']
    title += ' '
    title += cards['Brand']
    
    
    
    #logic to fit title in ebay listing, change as desired
    
    if len(title) > 80:
        title.replace("POKEMON","")
        
    if len(title) < 80: 
        title += " " +  cards['Card Number']
        
    if len(title) > 80:
        title.replace(cards['Card Number'], "")
        
    return {"title": title, "grade": grade, "front": cards['Front'], "back" : cards['Back']}



get_listing_title(70952644)



In [ ]:
from ebaysdk.trading import Connection
import ebaysdk.shopping


#https://developer.ebay.com/
app_id = 'INSERT APP_ID HERE'


'''
@param cert: cert number 
@param price: listing price BIN
@param min_offer_price (optional) offer threshold 

reads uses the app_id of your ebay account and lists to your ebay account.

requires step 2-3 of readme at top to be completed

todo: figure out how to add shipping options that aren't free

'''

def list_card(cert, price=300.00,min_offer_price = int(price*.5)):
    
    listing_info = get_listing_title(cert)
    title = listing_info['title']
    if len(title) > 79: 
        title = title[0:79]
    
    title = title.replace('-', ' ')
    
    front = listing_info['front']
    back = listing_info['back']
    grade = listing_info['grade']
    
    api = Connection(appid= app_id, config_file='ebay.yaml', domain="api.ebay.com")                  
                               
        
    
    request = {
        "Item": {
            "Title": title.replace("&", "").replace("  ", " ").replace("/", " ") ,
            "Country": "US",
            "PostalCode": "94303",
            "Site": "US",
            "ConditionID": 3000,
            "PrimaryCategory": {"CategoryID": "183454"},
            "Description": title.replace("&", "").replace("  ", " ").replace("/", " ") + " Ships from USA. Will be shipped in bubble mailer with protection. Thank you for browsing my ebay store! Sales are Final. ",
            "ListingType": "FixedPriceItem",
            "ListingDuration": "Days_30",
            'StartPrice': price,
            'eBayPlusEligible': 'false',
            'GlobalShipping': 'true',
            
            'PictureDetails': {
            'GalleryType': 'Plus',
            'PhotoDisplay': 'SuperSize',
            'ExternalPictureURL': [front,back]
        },
            "BestOfferDetails":{
                "BestOfferEnabled":"True",
                "MinimumBestOfferPrice": min_offer_price
            },
            
            
            "ItemSpecifics":{
                "NameValueList": [
                    {"Name": "Grade", "Value": str(grade)},
                    {"Name": "Graded", "Value": "Yes"},
                    {"Name": "Game", "Value": "	Pokémon TCG"}
                ]
            },
            
            
            'DispatchTimeMax':5,
            "Currency": "USD",
            "ReturnPolicy": {
                "ReturnsAcceptedOption": "ReturnsNotAccepted",
                "Description": "Sales are Final",
            },
            "ShippingDetails": {
                "ShippingServiceOptions": {
                    'GlobalShipping': 'True',
                    "FreeShipping": "True",
                    "ShippingService": "USPSFirstClass"
                }
            }
            
        }
    }

    api.execute("AddFixedPriceItem", request)
    


In [ ]:

#FOLLOWING CODE EXTRACTS CERTS FROM an IMAGE

from PIL import Image
import pytesseract
import re 

def extract_cert(image):
    result = pytesseract.image_to_string(Image.open(image))
    cert = re.findall(r'(\d{8})',result)
    if cert == []:
        return ''
    else:
        cert = cert[0]
        return cert
    
    
    
    